In [1]:
import json
import pandas as pd
from datetime import datetime

In [2]:
with open("./query.json", 'r') as f:
    data = json.load(f)['data']['swaps']
for d in data:
    d["transaction_blockNumber"] = d["transaction"]["blockNumber"]
    del d["transaction"]

In [3]:
swap_fields = [field for field in data[0].keys() if field != "pair"]
token_fields = []
for i in range(2):
    token_fields.extend([f"token{i}_{field}" for field in data[0]["pair"][f"token{i}"] if field not in ["id", "symbol"]])
token_fields.extend([field for field in data[0]["pair"].keys() if "token" not in field])

In [4]:
rawdf = {field: [] for field in swap_fields}
for field in token_fields:
    rawdf[field] = []
    
for d in data:
    for field in rawdf.keys():
        if field in token_fields:
            if "token" in field:
                token, token_field = field.split('_')
                rawdf[field].append(d["pair"][token][token_field])
            else:
                rawdf[field].append(d["pair"][field])
        else:
            rawdf[field].append(d[field])
rawdf = pd.DataFrame.from_dict(rawdf)

In [5]:
symbols = [data[0]["pair"][f"token{i}"]["symbol"] for i in range(2)]

rename_cols = {}
for col in rawdf.columns:
    if "amount" in col:
        idx = int(col[col.find("amount") + 6])
        rename_cols[col] = col.replace(f"amount{idx}", f"{symbols[idx]}_")
    if "token" in col:
        idx = int(col[col.find("token") + 5])
        rename_cols[col] = col.replace(f"token{idx}", f"{symbols[idx]}")
rawdf = rawdf.rename(rename_cols, axis=1)

In [6]:
df = rawdf.copy()
for col in rawdf.columns:
    if col != "id":
        df[col] = rawdf[col].astype(float)
    else:
        df[col] = rawdf[col].astype("string")

In [7]:
side = ["Sell" if df.loc[i, "UNI_In"] > 0 else "Buy" for i in range(df.shape[0])]
df["Side"] = pd.Series(side, dtype="string")

In [8]:
cols = {
    "timestamp": "Timestamp",
    "Side": "Side",
    "UNI_tradeVolume": "Base Currency Vol.",
    "WETH_tradeVolume": "Quote Currency Vol.",
    "volumeUSD": "Vol. (USD)",
    "totalSupply": "Liquidity in Pool",
    "id": "PoolID"
}
df = df.rename(cols, axis=1)
df = df[[colname for _, colname in cols.items()]]

In [9]:
for i in range(df.shape[0]):
    df.loc[i, "Timestamp"] = datetime.utcfromtimestamp(int(df.loc[i, "Timestamp"])).strftime('%Y-%m-%d %H:%M:%S')

In [11]:
df.head()

,Timestamp,Side,Base Currency Vol.,Quote Currency Vol.,Vol. (USD),Liquidity in Pool,PoolID
0,2021-06-18 07:32:19,Sell,7.641089e+08,2.432909e+08,6.632907e+09,104814.119473,0x839549af1837381eb2f13a49fb506a036f0aa5f57577...
1,2021-06-18 07:32:19,Buy,7.641089e+08,2.432909e+08,6.632907e+09,104814.119473,0x069f018c8599612d3f2a9d5c30cd4ecbeefd8f3c3fbf...
2,2021-06-18 07:29:54,Sell,7.641089e+08,2.432909e+08,6.632907e+09,104814.119473,0xf1aed41c1b14097be325a6e423afe663c2e05c5bfc5b...
3,2021-06-18 07:29:39,Sell,7.641089e+08,2.432909e+08,6.632907e+09,104814.119473,0xc97e17874621de8499c3a5306803e2a1d0341a959e6e...
4,2021-06-18 07:09:54,Buy,7.641089e+08,2.432909e+08,6.632907e+09,104814.119473,0xf4497291493973ee75040f235e37be2c5ddfe0bba420...
